## stack overflow intersection circles method

In [1]:
'''
FINDING THE INTERSECTION COORDINATES (LAT/LON) OF TWO CIRCLES (GIVEN THE COORDINATES OF THE CENTER AND THE RADII)

Many thanks to Ture Pålsson who directed me to the right source, the code below is based on whuber's brilliant logic and
explanation here https://gis.stackexchange.com/questions/48937/calculating-intersection-of-two-circles 

The idea is that;
  1. The points in question are the mutual intersections of three spheres: a sphere centered beneath location x1 (on the 
  earth's surface) of a given radius, a sphere centered beneath location x2 (on the earth's surface) of a given radius, and
  the earth itself, which is a sphere centered at O = (0,0,0) of a given radius.
  2. The intersection of each of the first two spheres with the earth's surface is a circle, which defines two planes.
  The mutual intersections of all three spheres therefore lies on the intersection of those two planes: a line.
  Consequently, the problem is reduced to intersecting a line with a sphere.

Note that "Decimal" is used to have higher precision which is important if the distance between two points are a few
meters.
'''
from decimal import Decimal
from math import cos, sin, sqrt
import math
import numpy as np

def intersection(p1, r1_meter, p2, r2_meter):
    # p1 = Coordinates of Point 1: latitude, longitude. This serves as the center of circle 1. Ex: (36.110174,  -90.953524)
    # r1_meter = Radius of circle 1 in meters
    # p2 = Coordinates of Point 2: latitude, longitude. This serves as the center of circle 1. Ex: (36.110174,  -90.953524)
    # r2_meter = Radius of circle 2 in meters
    '''
    1. Convert (lat, lon) to (x,y,z) geocentric coordinates.
    As usual, because we may choose units of measurement in which the earth has a unit radius
    '''
    x_p1 = Decimal(cos(math.radians(p1[1]))*cos(math.radians(p1[0])))  # x = cos(lon)*cos(lat)
    y_p1 = Decimal(sin(math.radians(p1[1]))*cos(math.radians(p1[0])))  # y = sin(lon)*cos(lat)
    z_p1 = Decimal(sin(math.radians(p1[0])))                           # z = sin(lat)
    x1 = (x_p1, y_p1, z_p1)

    x_p2 = Decimal(cos(math.radians(p2[1]))*cos(math.radians(p2[0])))  # x = cos(lon)*cos(lat)
    y_p2 = Decimal(sin(math.radians(p2[1]))*cos(math.radians(p2[0])))  # y = sin(lon)*cos(lat)
    z_p2 = Decimal(sin(math.radians(p2[0])))                           # z = sin(lat)
    x2 = (x_p2, y_p2, z_p2)
    '''
    2. Convert the radii r1 and r2 (which are measured along the sphere) to angles along the sphere.
    By definition, one nautical mile (NM) is 1/60 degree of arc (which is pi/180 * 1/60 = 0.0002908888 radians).
    '''
    r1 = Decimal(math.radians((r1_meter/1852) / 60)) # r1_meter/1852 converts meter to Nautical mile.
    r2 = Decimal(math.radians((r2_meter/1852) / 60))
    '''
    3. The geodesic circle of radius r1 around x1 is the intersection of the earth's surface with an Euclidean sphere
    of radius sin(r1) centered at cos(r1)*x1.

    4. The plane determined by the intersection of the sphere of radius sin(r1) around cos(r1)*x1 and the earth's surface
    is perpendicular to x1 and passes through the point cos(r1)x1, whence its equation is x.x1 = cos(r1)
    (the "." represents the usual dot product); likewise for the other plane. There will be a unique point x0 on the
    intersection of those two planes that is a linear combination of x1 and x2. Writing x0 = ax1 + b*x2 the two planar
    equations are;
       cos(r1) = x.x1 = (a*x1 + b*x2).x1 = a + b*(x2.x1)
       cos(r2) = x.x2 = (a*x1 + b*x2).x2 = a*(x1.x2) + b
    Using the fact that x2.x1 = x1.x2, which I shall write as q, the solution (if it exists) is given by
       a = (cos(r1) - cos(r2)*q) / (1 - q^2),
       b = (cos(r2) - cos(r1)*q) / (1 - q^2).
    '''
    q = Decimal(np.dot(x1, x2))

    if q**2 != 1 :
        a = (Decimal(cos(r1)) - Decimal(cos(r2))*q) / (1 - q**2)
        b = (Decimal(cos(r2)) - Decimal(cos(r1))*q) / (1 - q**2)
        '''
        5. Now all other points on the line of intersection of the two planes differ from x0 by some multiple of a vector
        n which is mutually perpendicular to both planes. The cross product  n = x1~Cross~x2  does the job provided n is 
        nonzero: once again, this means that x1 and x2 are neither coincident nor diametrically opposite. (We need to 
        take care to compute the cross product with high precision, because it involves subtractions with a lot of
        cancellation when x1 and x2 are close to each other.)
        '''
        n = np.cross(x1, x2)
        '''
        6. Therefore, we seek up to two points of the form x0 + t*n which lie on the earth's surface: that is, their length
        equals 1. Equivalently, their squared length is 1:  
        1 = squared length = (x0 + t*n).(x0 + t*n) = x0.x0 + 2t*x0.n + t^2*n.n = x0.x0 + t^2*n.n
        '''
        x0_1 = [a*f for f in x1]
        x0_2 = [b*f for f in x2]
        x0 = [sum(f) for f in zip(x0_1, x0_2)]
        '''
          The term with x0.n disappears because x0 (being a linear combination of x1 and x2) is perpendicular to n.
          The two solutions easily are   t = sqrt((1 - x0.x0)/n.n)    and its negative. Once again high precision
          is called for, because when x1 and x2 are close, x0.x0 is very close to 1, leading to some loss of
          floating point precision.
        '''
        if (np.dot(x0, x0) <= 1) & (np.dot(n,n) != 0): # This is to secure that (1 - np.dot(x0, x0)) / np.dot(n,n) > 0
            t = Decimal(sqrt((1 - np.dot(x0, x0)) / np.dot(n,n)))
            t1 = t
            t2 = -t

            i1 = x0 + t1*n
            i2 = x0 + t2*n
            '''
            7. Finally, we may convert these solutions back to (lat, lon) by converting geocentric (x,y,z) to geographic
            coordinates. For the longitude, use the generalized arctangent returning values in the range -180 to 180
            degrees (in computing applications, this function takes both x and y as arguments rather than just the
            ratio y/x; it is sometimes called "ATan2").
            '''

            i1_lat = math.degrees( math.asin(i1[2]))
            i1_lon = math.degrees( math.atan2(i1[1], i1[0] ) )
            ip1 = (i1_lat, i1_lon)

            i2_lat = math.degrees( math.asin(i2[2]))
            i2_lon = math.degrees( math.atan2(i2[1], i2[0] ) )
            ip2 = (i2_lat, i2_lon)
            return [ip1, ip2]
        elif (np.dot(n,n) == 0):
            return("The centers of the circles can be neither the same point nor antipodal points.")
        else:
            return("The circles do not intersect")
    else:
        return("The centers of the circles can be neither the same point nor antipodal points.")

'''
Example: the output of below is  [(36.989311051533505, -88.15142628069133), (38.2383796094578, -92.39048549120287)]
         intersection_points = intersection((37.673442, -90.234036), 107.5*1852, (36.109997, -90.953669), 145*1852)
         print(intersection_points)
'''


'\nExample: the output of below is  [(36.989311051533505, -88.15142628069133), (38.2383796094578, -92.39048549120287)]\n         intersection_points = intersection((37.673442, -90.234036), 107.5*1852, (36.109997, -90.953669), 145*1852)\n         print(intersection_points)\n'

In [2]:
def polygon_area(points):
    poly_area = 0

    count = len(points)
    j = count - 1

    if count < 3:
        return None

    for i in range(0, count):
        p1_x, p1_y = points[i]
        p2_x, p2_y = points[j]

        poly_area += p1_x * p2_y
        poly_area -= p1_y * p2_x
        j = i

    poly_area /= 2
    if np.isnan(poly_area):
        return None

    return abs(poly_area)


def polygon_centroid(points):
    f_total = 0
    x_total = 0
    y_total = 0

    count = len(points)
    j = count - 1

    if count < 3:
        return None

    for i in range(0, count):
        p1_x, p1_y = points[i]
        p2_x, p2_y = points[j]

        f_total = p1_x * p2_y - p2_x * p1_y
        x_total += (p1_x + p2_x) * f_total
        y_total += (p1_y + p2_y) * f_total
        j = i

    six_area = polygon_area(points) * 6
    if six_area is None:
        return None

    print(six_area)

    return x_total / six_area, y_total / six_area, six_area

In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from math import radians, cos, sin, asin, sqrt, pi
from utils.helpers import circle_intersections, rtt_to_km

probe_circles = {}
probe_circles["a"] = (48.68559000514443, 1.4890742633310852, 4.5, None, None)
probe_circles["b"] = (49.386153229694834, 2.2976143160251232, 4.5, None, None)
probe_circles["c"] = (48.86168126136162, 2.916765060171397, 4.5, None, None)

test_circles = {}
for key, (lat, lon, rtt, _, _) in probe_circles.items():
    d = rtt_to_km(rtt,  4/9)
    test_circles[key] = (lat, lon, d)

# CONCLUSION: if we take one working example and use it for calculation
# results differs from reality
# noethless, program seems to work otherwise

intersections = circle_intersections(probe_circles.values(), speed_threshold=4/9)

centroid = polygon_centroid(intersections)

print("intersecion:", intersections)
print("centroid:", centroid)
print("polygon:")

df_circles = pd.DataFrame({
    'Latitude': np.array([lat_long[0] for lat_long in test_circles.values()]),
    'Longitude': np.array([lat_long[1] for lat_long in test_circles.values()]),
    'Radius': np.array([lat_long[2] for lat_long in test_circles.values()]),
})

fig_map3 = px.scatter_mapbox(df_circles['Radius'], lon=df_circles['Longitude'], lat=df_circles['Latitude'],
                             hover_name='Radius', zoom=9, width=300, height=500)

# # parameters
N = 360 # number of discrete sample points to be generated along the circle

# generate points
circles = []
for i, (index, row) in enumerate(df_circles.iterrows()):
    circle_lats, circle_lons = [], []

    lat = df_circles['Latitude'][i]
    lon = df_circles['Longitude'][i]
    r = df_circles['Radius'][i]

    print(lat, lon, r)

    for k in range(N):
        # compute
        angle = pi*2*k/N
        dx = r*1000*cos(angle)
        dy = r*1000*sin(angle)
        circle_lats.append(lat + (180/pi)*(dy/6378137))
        circle_lons.append(lon + (180/pi)*(dx/6378137)/cos(lat*pi/180))

    circle_lats.append(circle_lats[0])
    circle_lons.append(circle_lons[0])

    fig_map3.add_trace(go.Scattermapbox(
        lat=circle_lats,
        lon=circle_lons,
        mode='lines',
        marker=go.scattermapbox.Marker(
            size=1, color="BlueViolet"
        ),
    ))

# add calculated intersections
print("calculated intersections:")
for lat, lon in intersections:
    print(lat, lon)
    fig_map3.add_trace(go.Scattermapbox(
        lat=[lat],
        lon=[lon],
        marker=go.scattermapbox.Marker(
            size=10, color="Red"
        ),
    ))

    print(lat, lon)

fig_map3.update_layout(mapbox_style='open-street-map', margin={'r':0, 't':0, 'l':0, 'b':0}, width=500)

fig_map3.show()

65.93339789053229
intersecion: [(47.386957237672654, 5.0264582064712995), (47.443964357438034, -0.5246397788863818), (51.38318224021115, 1.4192374833462051)]
centroid: (-48.738034611773934, -1.9736853036437085, 65.93339789053229)
polygon:
48.68559000514443 1.4890742633310852 300.0
49.386153229694834 2.2976143160251232 300.0
48.86168126136162 2.916765060171397 300.0
calculated intersections:
47.386957237672654 5.0264582064712995
47.386957237672654 5.0264582064712995
47.443964357438034 -0.5246397788863818
47.443964357438034 -0.5246397788863818
51.38318224021115 1.4192374833462051
51.38318224021115 1.4192374833462051


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
from analysis.helpers import rtt_to_km, circle_intersections, polygon_centroid, haversine, circle_preprocessing


probe_circles = {}
probe_circles["a"] = (48.68559000514443, 1.4890742633310852, 6, None, None)
probe_circles["b"] = (49.386153229694834, 2.2976143160251232, 6, None, None)
probe_circles["c"] = (48.86168126136162, 2.916765060171397, 6, None, None)

test = {}
for key, (lat, lon, rtt, _, _) in probe_circles.items():
    d = rtt_to_km(rtt,  4/9)
    r = d / 2
    test[key] = (lat, lon, d)
# CONCLUSION: if we take one working example and use it for calculation
# results differs from reality
# noethless, program seems to work otherwise

for ip, circle in probe_circles.items():
    print(ip, circle)

intersections = circle_intersections(probe_circles.values(), speed_threshold=4/9)
print("intersecion:", intersections)

for i, intersection in enumerate(intersections):
    test[str(i)] = (intersection[0], intersection[1], 10)
print_circles(test)

a (48.68559000514443, 1.4890742633310852, 6, None, None)
b (49.386153229694834, 2.2976143160251232, 6, None, None)
c (48.86168126136162, 2.916765060171397, 6, None, None)
intersecion: [(46.80516379449255, 6.051286722446926), (46.85777327653742, -1.5374828427557328), (52.27563699722426, 1.1302907170221195)]


NameError: name 'print_circles' is not defined

In [ ]:
import numpy as np
import itertools

from math import radians, cos, sin, asin, sqrt

def internet_speed(rtt, speed_threshold):
    if speed_threshold is not None:
        return speed_threshold

    if rtt >= 80:
        speed_threshold = 4 / 9
    if rtt >= 5 and rtt < 80:
        speed_threshold = 3 / 9
    if rtt >= 0 and rtt < 5:
        speed_threshold = 1 / 6

    return speed_threshold


def rtt_to_km(rtt, speed_threshold=None, c=300):
    return internet_speed(rtt, speed_threshold) * rtt * c


def is_within_cirle(vp_geo, rtt, candidate_geo, speed_threshold=None):
    d = rtt_to_km(rtt, speed_threshold)
    d_vp_candidate = haversine(vp_geo, candidate_geo)
    if d < d_vp_candidate:
        return False
    else:
        return True


def geo_to_cartesian(lat, lon):
    lat *= np.pi / 180
    lon *= np.pi / 180

    x = np.cos(lon) * np.cos(lat)
    y = np.sin(lon) * np.cos(lat)
    z = np.sin(lat)

    return x, y, z


def check_circle_inclusion(c_1, c_2):
    lat_1, lon_1, rtt_1, d_1, r_1 = c_1
    lat_2, lon_2, rtt_2, d_2, r_2 = c_2
    d = haversine((lat_1, lon_1), (lat_2, lon_2))
    #print(f"distance between: d1 :{round(d_1,3)} / d2: {round(d_2,3)}  : {d}")
    if d_1 > (d + d_2):
        #print(f"remove c1 keep c2")
        return c_1, c_2
    elif d_2 > (d + d_1):
        #print(f"remove c1 keep c2")
        return c_2, c_1
    return None, None


def circle_preprocessing(circles, speed_threshold=None):
    circles_to_ignore = set()
    circles_to_keep = set()
    for c_1, c_2 in itertools.combinations(circles, 2):
        lat_1, lon_1, rtt_1, d_1, r_1 = c_1
        if d_1 is None:
            d_1 = rtt_to_km(rtt_1, speed_threshold)
        if r_1 is None:
            r_1 = d_1 / 6371

        lat_2, lon_2, rtt_2, d_2, r_2 = c_2
        if d_2 is None:
            d_2 = rtt_to_km(rtt_2, speed_threshold)
        if r_2 is None:
            r_2 = d_2 / 6371

        remove, keep = check_circle_inclusion(
            (lat_1, lon_1, rtt_1, d_1, r_1), (lat_2, lon_2, rtt_2, d_2, r_2)
        )

        # if remove:
        #     circles_to_ignore.add(remove)
        #     circles_to_keep.add(keep)
        # else:
        circles_to_keep.add((lat_1, lon_1, rtt_1, d_1, r_1))
        circles_to_keep.add((lat_2, lon_2, rtt_2, d_2, r_2))

    return circles_to_keep - circles_to_ignore


def circle_intersections(circles, speed_threshold=None):
    intersect_points = []

    circles = circle_preprocessing(circles, speed_threshold=speed_threshold)

    if len(circles) <= 2:
        print(f"Not enough circles ({len(circles)}).")
        return [], circles

    for c_1, c_2 in itertools.combinations(circles, 2):
        lat_1, lon_1, rtt_1, d_1, r_1 = c_1
        lat_2, lon_2, rtt_2, d_2, r_2 = c_2

        # print("circle:", lat_1, lon_1, rtt_1, d_1, r_1)
        # print("circle:", lat_2, lon_2, rtt_2, d_2, r_2)

        x1 = np.array(list(geo_to_cartesian(lat_1, lon_1)))
        x2 = np.array(list(geo_to_cartesian(lat_2, lon_2)))

        q = np.dot(x1, x2)

        a = (np.cos(r_1) - np.cos(r_2) * q) / (1 - (q ** 2))
        b = (np.cos(r_2) - np.cos(r_1) * q) / (1 - (q ** 2))

        x0 = a * x1 + b * x2

        n = np.cross(x1, x2)
        if (1 - np.dot(x0, x0)) / np.dot(n, n) <= 0:
            print("ANYCAST???", (lat_1, lon_1, rtt_1, d_1), (lat_2, lon_2, rtt_2, d_2))
            continue

        t = np.sqrt((1 - np.dot(x0, x0)) / np.dot(n, n))

        i1 = x0 + t * n
        i2 = x0 - t * n

        i_lon_1 = np.arctan(i1[1] / i1[0]) / (np.pi / 180)
        i_lat_1 = np.arctan(i1[2] / np.sqrt((i1[0] ** 2) + (i1[1] ** 2))) / (
            np.pi / 180
        )
        intersect_points.append((i_lat_1, i_lon_1))

        i_lon_2 = np.arctan(i2[1] / i2[0]) / (np.pi / 180)
        i_lat_2 = np.arctan(i2[2] / np.sqrt((i2[0] ** 2) + (i2[1] ** 2))) / (
            np.pi / 180
        )
        intersect_points.append((i_lat_2, i_lon_2))

    filtred_points = []
    for point_geo in intersect_points:
        # for lat_c, long_c, rtt_c, d_c, r_c in circles:
        #     if not is_within_cirle((lat_c, long_c), rtt_c, point_geo, speed_threshold):
        #         print("point is not within circle")
        #         break
        # else:
        filtred_points.append(point_geo)

    return filtred_points, circles


def polygon_area(points):
    poly_area = 0

    count = len(points)
    j = count - 1

    if count < 3:
        return None

    for i in range(0, count):
        p1_x, p1_y = points[i]
        p2_x, p2_y = points[j]

        poly_area += p1_x * p2_y
        poly_area -= p1_y * p2_x
        j = i

    poly_area /= 2
    if np.isnan(poly_area):
        return None

    return abs(poly_area)


def polygon_centroid(points):
    f_total = 0
    x_total = 0
    y_total = 0

    count = len(points)
    j = count - 1

    if count < 3:
        return None

    for i in range(0, count):
        p1_x, p1_y = points[i]
        p2_x, p2_y = points[j]

        f_total = p1_x * p2_y - p2_x * p1_y
        x_total += (p1_x + p2_x) * f_total
        y_total += (p1_y + p2_y) * f_total
        j = i

    six_area = polygon_area(points) * 6
    if six_area is None:
        return None

    return x_total / six_area, y_total / six_area


def haversine(input_location, block_location):
    """Distance between two locations in earth."""
    in_lat, in_lon, block_lat, block_lon = map(
        np.radians, [*input_location, *block_location]
    )

    dlat = block_lat - in_lat
    dlon = block_lon - in_lon

    distances = (
        np.sin(dlat / 2.0) ** 2
        + np.cos(in_lat) * np.cos(block_lat) * np.sin(dlon / 2.0) ** 2
    )

    return 6367 * 2 * np.arcsin(np.sqrt(distances))

def distance(lat1, lat2, lon1, lon2):

    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    c = 2 * asin(sqrt(a))

    r = 6371

    return(c * r)
